In [6]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt

DATA_PATH = "data_10.json"


def load_data(data_path):
    """Loads training dataset from json file.
        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    return X, y


#def plot_history(history):
#    """Plots accuracy/loss for training/validation set as a function of the epochs
#        :param history: Training history of model
#        :return:
#    """

#    fig, axs = plt.subplots(2)

    # create accuracy sublpot
#    axs[0].plot(history.history["accuracy"], label="train accuracy")
#    axs[0].plot(history.history["val_accuracy"], label="test accuracy")
#    axs[0].set_ylabel("Accuracy")
#    axs[0].legend(loc="lower right")
#    axs[0].set_title("Accuracy eval")

    # create error sublpot
#    axs[1].plot(history.history["loss"], label="train error")
#    axs[1].plot(history.history["val_loss"], label="test error")
#    axs[1].set_ylabel("Error")
#    axs[1].set_xlabel("Epoch")
#    axs[1].legend(loc="upper right")
#    axs[1].set_title("Error eval")

#    plt.show()


def prepare_datasets(test_size, validation_size):
    """Loads data and splits it into train, validation and test sets.
    :param test_size (float): Value in [0, 1] indicating percentage of data set to allocate to test split
    :param validation_size (float): Value in [0, 1] indicating percentage of train set to allocate to validation split
    :return X_train (ndarray): Input training set
    :return X_validation (ndarray): Input validation set
    :return X_test (ndarray): Input test set
    :return y_train (ndarray): Target training set
    :return y_validation (ndarray): Target validation set
    :return y_test (ndarray): Target test set
    """

    # load data
    X, y = load_data(DATA_PATH)

    # create train, validation and test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # add an axis to input sets
    X_train = X_train[..., np.newaxis]
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation, y_test


def build_model(input_shape):
    """Generates CNN model
    :param input_shape (tuple): Shape of input set
    :return model: CNN model
    """

    # build network topology
    model = keras.Sequential()

    # 1st conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 2nd conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 3rd conv layer
    model.add(keras.layers.Conv2D(32, (2, 2), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # flatten output and feed it into dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))

    # output layer
    model.add(keras.layers.Dense(10, activation='softmax'))

    return model


def predict(model, X, y):
    """Predict a single sample using the trained model
    :param model: Trained classifier
    :param X: Input data
    :param y (int): Target
    """

    # add a dimension to input data for sample - model.predict() expects a 4d array in this case
    X = X[np.newaxis, ...] # array shape (1, 130, 13, 1)

    # perform prediction
    prediction = model.predict(X)

    # get index with max value
    predicted_index = np.argmax(prediction, axis=1)

    print("Target: {}, Predicted label: {}".format(y, predicted_index))


if __name__ == "__main__":

    # get train, validation, test splits
    X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)

    # create network
    input_shape = (X_train.shape[1], X_train.shape[2], 1)
    model = build_model(input_shape)

    # compile model
    optimiser = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimiser,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.summary()

    # train model
    history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=1000)

    # plot accuracy/error for training and validation
    #plot_history(history)

    # evaluate model on test set
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    print('\nTest accuracy:', test_acc)

    # pick a sample to predict from the test set
    X_to_predict = X_test[100]
    y_to_predict = y_test[100]

    # predict sample
    predict(model, X_to_predict, y_to_predict)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 20, 11, 32)        320       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 10, 6, 32)         0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 10, 6, 32)         128       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 8, 4, 32)          9248      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 4, 2, 32)          0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 4, 2, 32)          128       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 3, 1, 32)         

1224/1224 [==============================] - 2s 2ms/sample - loss: 1.2328 - acc: 0.5980 - val_loss: 1.3587 - val_acc: 0.5425
Epoch 45/1000
1224/1224 [==============================] - 2s 2ms/sample - loss: 1.2237 - acc: 0.5874 - val_loss: 1.3761 - val_acc: 0.5490
Epoch 46/1000
1224/1224 [==============================] - 2s 2ms/sample - loss: 1.2240 - acc: 0.6054 - val_loss: 1.3503 - val_acc: 0.5490
Epoch 47/1000
1224/1224 [==============================] - 2s 2ms/sample - loss: 1.2264 - acc: 0.5923 - val_loss: 1.3325 - val_acc: 0.5490
Epoch 48/1000
1224/1224 [==============================] - 2s 2ms/sample - loss: 1.1971 - acc: 0.6021 - val_loss: 1.3452 - val_acc: 0.5294
Epoch 49/1000
1224/1224 [==============================] - 2s 2ms/sample - loss: 1.1854 - acc: 0.6070 - val_loss: 1.3239 - val_acc: 0.5392
Epoch 50/1000
1224/1224 [==============================] - 2s 2ms/sample - loss: 1.1730 - acc: 0.6225 - val_loss: 1.3224 - val_acc: 0.5654
Epoch 51/1000
1224/1224 [================

1224/1224 [==============================] - 3s 2ms/sample - loss: 0.8281 - acc: 0.7263 - val_loss: 1.1804 - val_acc: 0.6144
Epoch 104/1000
1224/1224 [==============================] - 3s 3ms/sample - loss: 0.8208 - acc: 0.7361 - val_loss: 1.1565 - val_acc: 0.6242
Epoch 105/1000
1224/1224 [==============================] - 3s 2ms/sample - loss: 0.7816 - acc: 0.7426 - val_loss: 1.1729 - val_acc: 0.6209
Epoch 106/1000
1224/1224 [==============================] - 2s 2ms/sample - loss: 0.7925 - acc: 0.7500 - val_loss: 1.1723 - val_acc: 0.5948
Epoch 107/1000
1224/1224 [==============================] - 2s 2ms/sample - loss: 0.8080 - acc: 0.7222 - val_loss: 1.1927 - val_acc: 0.6078
Epoch 108/1000
1224/1224 [==============================] - 3s 2ms/sample - loss: 0.7901 - acc: 0.7353 - val_loss: 1.1793 - val_acc: 0.6176
Epoch 109/1000
1224/1224 [==============================] - 2s 2ms/sample - loss: 0.8080 - acc: 0.7361 - val_loss: 1.1782 - val_acc: 0.5948
Epoch 110/1000
1224/1224 [=========

1224/1224 [==============================] - 3s 2ms/sample - loss: 0.5664 - acc: 0.8203 - val_loss: 1.1200 - val_acc: 0.6471
Epoch 162/1000
1224/1224 [==============================] - 2s 2ms/sample - loss: 0.5694 - acc: 0.8170 - val_loss: 1.1379 - val_acc: 0.6209
Epoch 163/1000
1224/1224 [==============================] - ETA: 0s - loss: 0.5687 - acc: 0.8018- ETA: 0s - loss: 0.5590 - acc: 0 - 2s 2ms/sample - loss: 0.5736 - acc: 0.7998 - val_loss: 1.1031 - val_acc: 0.6503
Epoch 164/1000
1224/1224 [==============================] - 2s 2ms/sample - loss: 0.5635 - acc: 0.8162 - val_loss: 1.1561 - val_acc: 0.6111
Epoch 165/1000
1224/1224 [==============================] - 2s 2ms/sample - loss: 0.5411 - acc: 0.8243 - val_loss: 1.1372 - val_acc: 0.6242
Epoch 166/1000
1224/1224 [==============================] - 3s 2ms/sample - loss: 0.5561 - acc: 0.8145 - val_loss: 1.1486 - val_acc: 0.6078
Epoch 167/1000
1224/1224 [==============================] - 3s 2ms/sample - loss: 0.5361 - acc: 0.8423 

1224/1224 [==============================] - 2s 2ms/sample - loss: 0.4179 - acc: 0.8611 - val_loss: 1.1523 - val_acc: 0.6275
Epoch 220/1000
1224/1224 [==============================] - 2s 2ms/sample - loss: 0.4138 - acc: 0.8734 - val_loss: 1.1560 - val_acc: 0.6275
Epoch 221/1000
1224/1224 [==============================] - 2s 2ms/sample - loss: 0.4209 - acc: 0.8676 - val_loss: 1.2056 - val_acc: 0.6176
Epoch 222/1000
1224/1224 [==============================] - 2s 2ms/sample - loss: 0.4077 - acc: 0.8734 - val_loss: 1.1654 - val_acc: 0.6405
Epoch 223/1000
1224/1224 [==============================] - 2s 2ms/sample - loss: 0.3979 - acc: 0.8742 - val_loss: 1.1652 - val_acc: 0.6242
Epoch 224/1000
1224/1224 [==============================] - 3s 3ms/sample - loss: 0.3885 - acc: 0.8775 - val_loss: 1.1648 - val_acc: 0.6373
Epoch 225/1000
1224/1224 [==============================] - 2s 2ms/sample - loss: 0.3713 - acc: 0.8979 - val_loss: 1.1444 - val_acc: 0.6340
Epoch 226/1000
1224/1224 [=========

...

1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0749 - acc: 0.9788 - val_loss: 1.8140 - val_acc: 0.6536
Epoch 742/1000
1224/1224 [==============================] - 1s 1ms/sample - loss: 0.0657 - acc: 0.9837 - val_loss: 1.8962 - val_acc: 0.6373
Epoch 743/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0750 - acc: 0.9812 - val_loss: 1.7088 - val_acc: 0.6634
Epoch 744/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0764 - acc: 0.9755 - val_loss: 1.8418 - val_acc: 0.6340
Epoch 745/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0637 - acc: 0.9812 - val_loss: 1.7595 - val_acc: 0.6111
Epoch 746/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0695 - acc: 0.9788 - val_loss: 1.9084 - val_acc: 0.6013
Epoch 747/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0634 - acc: 0.9788 - val_loss: 1.8331 - val_acc: 0.6373
Epoch 748/1000
1224/1224 [=========

1224/1224 [==============================] - 1s 1ms/sample - loss: 0.0552 - acc: 0.9837 - val_loss: 1.8997 - val_acc: 0.6373
Epoch 800/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0563 - acc: 0.9812 - val_loss: 1.8052 - val_acc: 0.6471
Epoch 801/1000
1224/1224 [==============================] - 1s 1ms/sample - loss: 0.0600 - acc: 0.9788 - val_loss: 1.8471 - val_acc: 0.6438
Epoch 802/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0708 - acc: 0.9796 - val_loss: 1.8486 - val_acc: 0.6438
Epoch 803/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0516 - acc: 0.9861 - val_loss: 1.8002 - val_acc: 0.6307
Epoch 804/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0522 - acc: 0.9869 - val_loss: 1.8937 - val_acc: 0.6242
Epoch 805/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0715 - acc: 0.9804 - val_loss: 1.8171 - val_acc: 0.6307
Epoch 806/1000
1224/1224 [=========

1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0796 - acc: 0.9730 - val_loss: 1.8930 - val_acc: 0.6307
Epoch 858/1000
1224/1224 [==============================] - 1s 1ms/sample - loss: 0.0531 - acc: 0.9902 - val_loss: 1.8384 - val_acc: 0.6373
Epoch 859/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0472 - acc: 0.9845 - val_loss: 1.8708 - val_acc: 0.6275
Epoch 860/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0625 - acc: 0.9812 - val_loss: 1.8572 - val_acc: 0.6307
Epoch 861/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0380 - acc: 0.9902 - val_loss: 1.8989 - val_acc: 0.6176
Epoch 862/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0691 - acc: 0.9820 - val_loss: 1.8844 - val_acc: 0.6373
Epoch 863/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0612 - acc: 0.9804 - val_loss: 1.7484 - val_acc: 0.6569
Epoch 864/1000
1224/1224 [=========

1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0701 - acc: 0.9747 - val_loss: 1.9489 - val_acc: 0.6340
Epoch 916/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0717 - acc: 0.9739 - val_loss: 1.8031 - val_acc: 0.6373
Epoch 917/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0550 - acc: 0.9820 - val_loss: 1.7824 - val_acc: 0.6536
Epoch 918/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0525 - acc: 0.9845 - val_loss: 1.8695 - val_acc: 0.6503
Epoch 919/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0546 - acc: 0.9828 - val_loss: 1.9747 - val_acc: 0.6340
Epoch 920/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0522 - acc: 0.9845 - val_loss: 1.9089 - val_acc: 0.6275
Epoch 921/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0368 - acc: 0.9918 - val_loss: 1.9378 - val_acc: 0.6405
Epoch 922/1000
1224/1224 [=========

1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0574 - acc: 0.9837 - val_loss: 1.9912 - val_acc: 0.6275
Epoch 974/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0632 - acc: 0.9779 - val_loss: 2.0322 - val_acc: 0.6471
Epoch 975/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0708 - acc: 0.9714 - val_loss: 2.0021 - val_acc: 0.6176
Epoch 976/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0484 - acc: 0.9861 - val_loss: 2.0999 - val_acc: 0.6275
Epoch 977/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0471 - acc: 0.9869 - val_loss: 1.8939 - val_acc: 0.6340
Epoch 978/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0654 - acc: 0.9796 - val_loss: 1.8919 - val_acc: 0.6340
Epoch 979/1000
1224/1224 [==============================] - 2s 1ms/sample - loss: 0.0529 - acc: 0.9788 - val_loss: 1.9149 - val_acc: 0.6634
Epoch 980/1000
1224/1224 [=========